In [5]:
%matplotlib inline
import pandas as pd
import bs4
from bs4 import BeautifulSoup
import numpy as np
import re
import matplotlib.pyplot as plt
import patsy
import statsmodels.api as sm
import scipy.stats as stats
from scipy.stats import ttest_ind, chisquare, normaltest

### Consolidating RMP with CAPE data
Now we want to put together our two dataframes into one, where each row represents a professor. Professors without a tid on RateMyProfessor will have to be excluded.

In [6]:
cape = pd.read_csv("capeReviewsCleaned2.csv")
cape

,Unnamed: 0,Instructor,tLname,tFname,tDept,Course,Term,Enroll,Evals Made,Eval / Enroll,Rcmnd Class,Rcmnd Instr,Study Hrs/wk,Avg Grade Expected,Avg Grade Received,tid
0,0,"Goldstein, Paul S",Goldstein,Paul,ANTH,ANTH 101 - Foundations/Social Complexity (A),183,137,73,0.532847,0.812,0.791,4.00,3.33,NaN,222495
1,1,"Semendeferi, Ekaterini",Semendeferi,Ekaterini,ANTH,ANTH 196A - Honors Studies in Anthropology (A),183,13,4,0.307692,1.000,1.000,7.00,4.00,NaN,1171683
2,2,"Rothschild, Amy C",Rothschild,Amy,ANTH,ANTH 21 - Race and Racisms (A),183,212,96,0.452830,0.947,0.978,3.01,3.58,NaN,-1
3,3,"Kang, Byung Chu",Kang,Byung,ANTH,ANTH 21 - Race and Racisms (B),183,314,214,0.681529,0.942,0.937,3.36,3.82,NaN,2308255
4,4,"Stewart, Alexander B",Stewart,Alexander,ANTH,ANTH 23 - Debating Multiculturalism (A),183,407,351,0.862408,0.879,0.956,3.53,3.58,NaN,2099749
5,5,"Rivera-Collazo, Isabel C",Rivera-Collazo,Isabel,ANTH,ANTH 3 - Global Archaeology (A),183,66,47,0.712121,0.957,1.000,3.14,3.49,NaN,-1
6,6,"Semendeferi, Ekaterini",Semendeferi,Ekaterini,ANTH,ANTH 87 - Freshman Seminar (A),183,17,13,0.764706,1.000,1.000,0.50,4.00,NaN,1171683
7,7,"Liss, Brady James",Liss,Brady,ANTH,ANTH 101 - Foundations/Social Complexity (A),182,21,13,0.619048,1.000,1.000,3.83,3.17,NaN,-1
8,8,"Wilder, Linnea Lorene",Wilder,Linnea,ANTH,ANTH 102 - Humans Are Cultural Animals (A),182,38,13,0.342105,1.000,0.833,2.81,3.50,3.72,-1
9,9,"Hrvoj Mihic, Branka",Hrvoj Mihic,Branka,ANTH,ANTH 2 - Human Origins (A),182,21,7,0.333333,0.857,1.000,4.17,3.75,3.58,2191664


In [7]:
rmp = pd.read_csv("modifiedProfInfo.csv")
rmp

,tid,tLname,tFname,gender,tDept,attendance,helpCount,id,notHelpCount,quality,...,rEasy,rEasyString,rHelpful,rInterest,rOverall,rTextBookUse,rWouldTakeAgain,takenForCredit,teacherGrade,teacherRatingTags
0,2325936,Aamari,Eddie,1.0,Mathematics,Not Mandatory,4.0,30218337,1.0,good,...,4.0,4.0,4.0,NaN,4.0,1.0,1.0,1.0,NaN,"['Lecture heavy', 'Caring', 'Lots of homework']"
1,2325936,Aamari,Eddie,1.0,Mathematics,Not Mandatory,3.0,29721878,1.0,awesome,...,3.0,3.0,5.0,NaN,5.0,1.0,1.0,1.0,NaN,"['Lots of homework', 'Caring', 'Hilarious']"
2,2325936,Aamari,Eddie,1.0,Mathematics,Not Mandatory,1.0,29701355,0.0,average,...,4.0,4.0,3.0,NaN,3.0,1.0,1.0,1.0,NaN,"['Accessible outside class', 'Lots of homework..."
3,2325936,Aamari,Eddie,1.0,Mathematics,Not Mandatory,0.0,29682099,1.0,average,...,4.0,4.0,3.0,NaN,3.0,1.0,0.0,1.0,NaN,[]
4,2325936,Aamari,Eddie,1.0,Mathematics,Not Mandatory,8.0,29557088,4.0,awesome,...,3.0,3.0,5.0,NaN,5.0,1.0,1.0,1.0,NaN,"['Lots of homework', 'Hilarious', 'Caring']"
5,2325936,Aamari,Eddie,1.0,Mathematics,Not Mandatory,7.0,29288327,0.0,awesome,...,2.0,2.0,5.0,NaN,5.0,1.0,1.0,1.0,4.0,"['Amazing lectures', 'Accessible outside class..."
6,286439,Abarbanel,Henry,1.0,Physics,Not Mandatory,0.0,28955886,0.0,awful,...,5.0,5.0,1.0,NaN,1.0,0.0,0.0,1.0,NaN,"['Graded by few things', 'Tough grader']"
7,286439,Abarbanel,Henry,1.0,Physics,Not Mandatory,0.0,28730130,1.0,awful,...,5.0,5.0,1.0,NaN,1.0,0.0,0.0,1.0,3.0,"['Graded by few things', 'Lecture heavy']"
8,286439,Abarbanel,Henry,1.0,Physics,Mandatory,0.0,25895906,1.0,poor,...,4.0,4.0,1.0,2.0,1.5,0.0,NaN,1.0,NaN,[]
9,286439,Abarbanel,Henry,1.0,Physics,NaN,0.0,20353335,1.0,awesome,...,1.0,1.0,5.0,5.0,5.0,1.0,NaN,NaN,NaN,[]


In [8]:
rmp.columns

Index(['tid', 'tLname', 'tFname', 'gender', 'tDept', 'attendance', 'helpCount',
       'id', 'notHelpCount', 'quality', 'rClarity', 'rClass', 'rComments',
       'sentiment', 'sentimentValue', 'rDate', 'rEasy', 'rEasyString',
       'rHelpful', 'rInterest', 'rOverall', 'rTextBookUse', 'rWouldTakeAgain',
       'takenForCredit', 'teacherGrade', 'teacherRatingTags'],
      dtype='object')

In [9]:
columnList = ['tid', 'gender', 'tDept', 'Enroll', 'Evals Made', 'Eval/Enroll', 'Rcmnd Class', 
           'Rcmnd Instr', 'Study Hrs/wk', 'Avg Grade Expected', 'Avg Grade Received', 
           'rEasy', 'rHelpful', 'rInterest' , 'rOverall', 'rWouldTakeAgain', 'rmp Grade', 
           'sentimentValue', 'teacherRatingTags', 'rmp Evals/Enroll', 'rmp Evals']
df = pd.DataFrame(columns=columnList)
tidList = cape['tid'].unique() # get array of unique tid's
tidList = np.setdiff1d(tidList,-1) # remove -1 from array and put in numerical order
df

,tid,gender,tDept,Enroll,Evals Made,Eval/Enroll,Rcmnd Class,Rcmnd Instr,Study Hrs/wk,Avg Grade Expected,...,rEasy,rHelpful,rInterest,rOverall,rWouldTakeAgain,rmp Grade,sentimentValue,teacherRatingTags,rmp Evals/Enroll,rmp Evals


In [31]:
def flToStr(fl_in):
    return str(fl_in)

In [20]:
#gets weighted average of values of a column
def getWeightedAvg(col, tid, totEvals):
    totEvals = totEvals.values[0]
    
    for index, row in cape[(pd.isnull(cape[col])) & (cape['tid'] == tid)].iterrows(): #remove evals from total for null entries in col
        totEvals -= row['Evals Made']
    avg = 0.0
    evalsCheck = 0
    for index, row in cape[cape['tid'] == tid].iterrows():
        if(not pd.isnull(row[col])):
            evalsCheck += row['Evals Made']
            avg += row[col] * row['Evals Made'] / totEvals # add to weighted avg
    
    if(evalsCheck != totEvals): #check to see if evals were added correctly
        print("EVALS CALCULATION ERROR: evalsCheck == " + evalsCheck + " , totEvals == " + totEvals)
        
    if(avg == 0): #return NaN if nothing was added to avg
        return np.nan
    
    return avg

In [22]:
getWeightedAvg('Avg Grade Expected', tid, df[df['tid'] == 26354]['Evals Made'])

,tid,gender,tDept,Enroll,Evals Made,Eval/Enroll,Rcmnd Class,Rcmnd Instr,Study Hrs/wk,Avg Grade Expected,Avg Grade Received,rEasy,rHelpful,rInterest,rOverall,rWouldTakeAgain,rmp Grade,sentimentValue,teacherRatingTags
6,26354,1,PSYC,121,85,0.702479,0.835,0.918,3.63,0,0,3.94595,2.97297,3.71429,3.17568,0,2,0.0698107,NaN


In [7]:
#get all unique rating tags for a tid
def getRatingTags(tid):
    tags = []
    for index, row in rmp[rmp['tid'] == tid].iterrows():
        tg = row['teacherRatingTags']
        if(not pd.isnull(tg)):
            tg = tg.strip('[ ]')
            tg = tg.replace('\'', ' ')
            tg = tg.split(',')
            tags.extend([a.strip() for a in tg if (a.strip() not in tags)])
    if('' in tags):    
        tags.remove('')
        
    tgstr = ""
    for tg in tags:
        tgstr += tg
        tgstr += ','
        
    return tgstr

Now we will iterate through each row and add values to each collumn from either rmp or capes. Since each row represents a unique professor, we must due some data processing on capes and rmp to consolidate multiple entries for each professor

In [25]:
counts = rmp.tid.value_counts()

for tid in tidList:
    index = df[df['tid'] == tid].index
    #print(tid)
    df.loc[index, 'gender'] = rmp[rmp['tid'] == str(tid)]['gender'].values[0]
    df.loc[index, 'tDept'] = cape[cape['tid'] == tid]['tDept'].values[0]
    df.loc[index, 'Enroll'] = cape[cape['tid'] == tid]['Enroll'].sum()
    df.loc[index, 'Evals Made'] = cape[cape['tid'] == tid]['Evals Made'].sum()
    df.loc[index, 'Eval/Enroll'] = df[df['tid'] == tid]['Evals Made'] / df[df['tid'] == tid]['Enroll']
    df.loc[index, 'Rcmnd Class'] = getWeightedAvg('Rcmnd Class', tid, df[df['tid'] == tid]['Evals Made'])
    df.loc[index, 'Rcmnd Instr'] = getWeightedAvg('Rcmnd Instr', tid, df[df['tid'] == tid]['Evals Made'])
    df.loc[index, 'Study Hrs/wk'] = getWeightedAvg('Study Hrs/wk', tid, df[df['tid'] == tid]['Evals Made'])
    df.loc[index, 'Avg Grade Expected'] = getWeightedAvg('Avg Grade Expected', tid, df[df['tid'] == tid]['Evals Made'])
    df.loc[index, 'Avg Grade Received'] = getWeightedAvg('Avg Grade Received', tid, df[df['tid'] == tid]['Evals Made'])
    df.loc[index, 'rEasy'] = rmp[rmp['tid'] == str(tid)]['rEasyString'].mean()
    df.loc[index, 'rHelpful'] = rmp[rmp['tid'] == str(tid)]['rHelpful'].mean()
    df.loc[index, 'rInterest'] = rmp[rmp['tid'] == str(tid)]['rInterest'].mean()
    df.loc[index, 'rOverall'] = rmp[rmp['tid'] == str(tid)]['rOverall'].mean()
    df.loc[index, 'rWouldTakeAgain'] = rmp[rmp['tid'] == str(tid)]['rWouldTakeAgain'].mean()
    df.loc[index, 'rmp Grade'] = rmp[rmp['tid'] == str(tid)]['teacherGrade'].mean()
    df.loc[index, 'sentimentValue'] = rmp[rmp['tid'] == str(tid)]['sentimentValue'].mean()
    df.loc[index, 'teacherRatingTags'] = getRatingTags(str(tid)) 
    df.loc[index, 'rmp Evals/Enroll'] = counts.get(str(tid)) / df[df['tid'] == tid]['Enroll']

df.to_csv("FullData.csv", index=False)
df

,tid,gender,tDept,Enroll,Evals Made,Eval/Enroll,Rcmnd Class,Rcmnd Instr,Study Hrs/wk,Avg Grade Expected,Avg Grade Received,rEasy,rHelpful,rInterest,rOverall,rWouldTakeAgain,rmp Grade,sentimentValue,teacherRatingTags
0,7853,1,PHYS,3803,1478,0.388641,0.717524,0.531486,6.2884,3.10071,2.89816,3.21795,2.46154,3.11628,2.47436,0.333333,3.48,0.0685425,NaN
1,7855,1,CHEM,19349,9774,0.505142,0.861172,0.885933,7.68978,3.01461,2.86043,3.71654,3.91601,3.2381,4.01837,0.730769,3.14286,0.140177,NaN
2,7857,1,MMW,2529,1579,0.624357,0.846111,0.947168,6.37059,3.31217,3.06766,2.88372,4.53488,3.50877,4.57558,NaN,3,0.287243,NaN
3,7858,1,MAE,3328,1333,0.400541,0.845998,0.716439,6.8405,3.4144,3.1106,2.86441,3.16949,3.22857,2.71186,0,4,0.0313841,NaN
4,7859,1,MATH,1958,851,0.434627,0.791921,0.683295,6.48363,3.28573,2.72818,2.82927,2.53659,3.16667,2.39024,0,NaN,0.0648914,NaN
5,9709,1,HUM,3744,1712,0.457265,0.675003,0.790459,6.61762,2.92455,2.72131,3.27586,3.44828,2.65217,3.25,0,2.66667,-0.00425588,NaN
6,26354,1,PSYC,121,85,0.702479,0.835,0.918,3.63,NaN,NaN,3.94595,2.97297,3.71429,3.17568,0,2,0.0698107,NaN
7,29553,1,ICAM,988,519,0.525304,0.707013,0.697555,4.2911,3.55841,3.18404,3.27273,1.72727,2.73333,1.70455,NaN,NaN,-0.0798961,NaN
8,32593,1,POLI,1308,519,0.396789,0.812904,0.763243,4.9153,3.34476,2.90283,2.5,3.64706,3.62963,3.98529,NaN,NaN,0.192362,NaN
9,35989,1,MMW,2535,1484,0.585404,0.728631,0.879964,6.20818,3.28512,3.15586,2.68519,4.01852,3.21053,3.89815,0.75,3.9,0.251128,NaN


In [10]:
df = pd.read_csv("FullData.csv")
df['rmp Evals'] = np.nan
df

,tid,gender,tDept,Enroll,Evals Made,Eval/Enroll,Rcmnd Class,Rcmnd Instr,Study Hrs/wk,Avg Grade Expected,...,rEasy,rHelpful,rInterest,rOverall,rWouldTakeAgain,rmp Grade,sentimentValue,teacherRatingTags,rmp Evals/Enroll,rmp Evals
0,7853,1.0,PHYS,3803,1478,0.388641,0.717524,0.531486,6.288403,3.100710,...,3.217949,2.461538,3.116279,2.474359,0.333333,3.480000,0.068542,"Get ready to read,Test heavy,Tough grader,",0.020510,NaN
1,7855,1.0,CHEM,19349,9774,0.505142,0.861172,0.885933,7.689782,3.014609,...,3.716535,3.916010,3.238095,4.018373,0.730769,3.142857,0.140177,"Amazing lectures,Caring,Tough grader,Respected...",0.019691,NaN
2,7857,1.0,MMW,2529,1579,0.624357,0.846111,0.947168,6.370589,3.312166,...,2.883721,4.534884,3.508772,4.575581,NaN,3.000000,0.287243,NaN,0.034006,NaN
3,7858,1.0,MAE,3328,1333,0.400541,0.845998,0.716439,6.840503,3.414405,...,2.864407,3.169492,3.228571,2.711864,0.000000,4.000000,0.031384,"Clear grading criteria,Graded by few things,Lo...",0.017728,NaN
4,7859,1.0,MATH,1958,851,0.434627,0.791921,0.683295,6.483631,3.285734,...,2.829268,2.536585,3.166667,2.390244,0.000000,NaN,0.064891,"Lecture heavy,Lots of homework,""Skip class? Yo...",0.020940,NaN
5,9709,1.0,HUM,3744,1712,0.457265,0.675003,0.790459,6.617617,2.924550,...,3.275862,3.448276,2.652174,3.250000,0.000000,2.666667,-0.004256,"Lecture heavy,Get ready to read,Amazing lectur...",0.015491,NaN
6,26354,1.0,PSYC,121,85,0.702479,0.835000,0.918000,3.630000,NaN,...,3.945946,2.972973,3.714286,3.175676,0.000000,2.000000,0.069811,"Tough grader,",0.305785,NaN
7,29553,1.0,ICAM,988,519,0.525304,0.707013,0.697555,4.291098,3.558410,...,3.272727,1.727273,2.733333,1.704545,NaN,NaN,-0.079896,NaN,0.022267,NaN
8,32593,1.0,POLI,1308,519,0.396789,0.812904,0.763243,4.915299,3.344759,...,2.500000,3.647059,3.629630,3.985294,NaN,NaN,0.192362,NaN,0.025994,NaN
9,35989,1.0,MMW,2535,1484,0.585404,0.728631,0.879964,6.208181,3.285121,...,2.685185,4.018519,3.210526,3.898148,0.750000,3.900000,0.251128,"Beware of pop quizzes,Accessible outside class...",0.021302,NaN


In [14]:
tidList = cape['tid'].unique() # get array of unique tid's
tidList = np.setdiff1d(tidList,-1) # remove -1 from array and put in numerical order
for tid in tidList:
    index = df[df['tid'] == tid].index
    df.loc[index, 'rmp Evals'] = len(rmp[rmp['tid'] == str(tid)])
    
df
    



,tid,gender,tDept,Enroll,Evals Made,Eval/Enroll,Rcmnd Class,Rcmnd Instr,Study Hrs/wk,Avg Grade Expected,...,rEasy,rHelpful,rInterest,rOverall,rWouldTakeAgain,rmp Grade,sentimentValue,teacherRatingTags,rmp Evals/Enroll,rmp Evals
0,7853,1.0,PHYS,3803,1478,0.388641,0.717524,0.531486,6.288403,3.100710,...,3.217949,2.461538,3.116279,2.474359,0.333333,3.480000,0.068542,"Get ready to read,Test heavy,Tough grader,",0.020510,78.0
1,7855,1.0,CHEM,19349,9774,0.505142,0.861172,0.885933,7.689782,3.014609,...,3.716535,3.916010,3.238095,4.018373,0.730769,3.142857,0.140177,"Amazing lectures,Caring,Tough grader,Respected...",0.019691,381.0
2,7857,1.0,MMW,2529,1579,0.624357,0.846111,0.947168,6.370589,3.312166,...,2.883721,4.534884,3.508772,4.575581,NaN,3.000000,0.287243,NaN,0.034006,86.0
3,7858,1.0,MAE,3328,1333,0.400541,0.845998,0.716439,6.840503,3.414405,...,2.864407,3.169492,3.228571,2.711864,0.000000,4.000000,0.031384,"Clear grading criteria,Graded by few things,Lo...",0.017728,59.0
4,7859,1.0,MATH,1958,851,0.434627,0.791921,0.683295,6.483631,3.285734,...,2.829268,2.536585,3.166667,2.390244,0.000000,NaN,0.064891,"Lecture heavy,Lots of homework,""Skip class? Yo...",0.020940,41.0
5,9709,1.0,HUM,3744,1712,0.457265,0.675003,0.790459,6.617617,2.924550,...,3.275862,3.448276,2.652174,3.250000,0.000000,2.666667,-0.004256,"Lecture heavy,Get ready to read,Amazing lectur...",0.015491,58.0
6,26354,1.0,PSYC,121,85,0.702479,0.835000,0.918000,3.630000,NaN,...,3.945946,2.972973,3.714286,3.175676,0.000000,2.000000,0.069811,"Tough grader,",0.305785,37.0
7,29553,1.0,ICAM,988,519,0.525304,0.707013,0.697555,4.291098,3.558410,...,3.272727,1.727273,2.733333,1.704545,NaN,NaN,-0.079896,NaN,0.022267,22.0
8,32593,1.0,POLI,1308,519,0.396789,0.812904,0.763243,4.915299,3.344759,...,2.500000,3.647059,3.629630,3.985294,NaN,NaN,0.192362,NaN,0.025994,34.0
9,35989,1.0,MMW,2535,1484,0.585404,0.728631,0.879964,6.208181,3.285121,...,2.685185,4.018519,3.210526,3.898148,0.750000,3.900000,0.251128,"Beware of pop quizzes,Accessible outside class...",0.021302,54.0


In [15]:
df.to_csv("FullData.csv", index=False)

In [27]:
df.to_csv("FullData.csv", index=False)


In [ ]:
tid = 26354
tid = str(tid)

rmp[rmp['tid'] == tid]

In [41]:
df.columns

Index(['tid', 'gender', 'tDept', 'Enroll', 'Evals Made', 'Eval/Enroll',
       'Rcmnd Class', 'Rcmnd Instr', 'Study Hrs/wk', 'Avg Grade Expected',
       'Avg Grade Received', 'rEasy', 'rHelpful', 'rInterest', 'rOverall',
       'rWouldTakeAgain', 'rmp Grade', 'sentimentValue', 'teacherRatingTags'],
      dtype='object')

In [24]:
tid = 26354
index = df[df['tid'] == tid].index
print(tid)
df.loc[index, 'gender'] = rmp[rmp['tid'] == str(tid)]['gender'].values[0]
df.loc[index, 'tDept'] = cape[cape['tid'] == tid]['tDept'].values[0]
df.loc[index, 'Enroll'] = cape[cape['tid'] == tid]['Enroll'].sum()
df.loc[index, 'Evals Made'] = cape[cape['tid'] == tid]['Evals Made'].sum()
df.loc[index, 'Eval/Enroll'] = df[df['tid'] == tid]['Evals Made'] / df[df['tid'] == tid]['Enroll']
df.loc[index, 'Rcmnd Class'] = getWeightedAvg('Rcmnd Class', tid, df[df['tid'] == tid]['Evals Made'])
df.loc[index, 'Rcmnd Instr'] = getWeightedAvg('Rcmnd Instr', tid, df[df['tid'] == tid]['Evals Made'])
df.loc[index, 'Avg Grade Expected'] = getWeightedAvg('Avg Grade Expected', tid, df[df['tid'] == tid]['Evals Made'])
df.loc[index, 'Avg Grade Received'] = getWeightedAvg('Avg Grade Received', tid, df[df['tid'] == tid]['Evals Made'])
df.loc[index, 'Study Hrs/wk'] = getWeightedAvg('Study Hrs/wk', tid, df[df['tid'] == tid]['Evals Made'])
df.loc[index, 'rEasy'] = rmp[rmp['tid'] == str(tid)]['rEasyString'].mean()
df.loc[index, 'rHelpful'] = rmp[rmp['tid'] == str(tid)]['rHelpful'].mean()
df.loc[index, 'rInterest'] = rmp[rmp['tid'] == str(tid)]['rInterest'].mean()
df.loc[index, 'rOverall'] = rmp[rmp['tid'] == str(tid)]['rOverall'].mean()
df.loc[index, 'rWouldTakeAgain'] = rmp[rmp['tid'] == str(tid)]['rWouldTakeAgain'].mean()
df.loc[index, 'rmp Grade'] = rmp[rmp['tid'] == str(tid)]['teacherGrade'].mean()
df.loc[index, 'sentimentValue'] = rmp[rmp['tid'] == str(tid)]['sentimentValue'].mean()
#df.loc[index, 'teacherRatingTags'] = getRatingTags(str(tid))

df[df['tid'] == tid]

26354


,tid,gender,tDept,Enroll,Evals Made,Eval/Enroll,Rcmnd Class,Rcmnd Instr,Study Hrs/wk,Avg Grade Expected,Avg Grade Received,rEasy,rHelpful,rInterest,rOverall,rWouldTakeAgain,rmp Grade,sentimentValue,teacherRatingTags
6,26354,1,PSYC,121,85,0.702479,0.835,0.918,3.63,NaN,NaN,3.94595,2.97297,3.71429,3.17568,0,2,0.0698107,NaN


In [22]:
counts = rmp.tid.value_counts()
rmp[rmp['tid'] == "1541248"]

,tid,tLname,tFname,gender,tDept,attendance,helpCount,id,notHelpCount,quality,...,rEasy,rEasyString,rHelpful,rInterest,rOverall,rTextBookUse,rWouldTakeAgain,takenForCredit,teacherGrade,teacherRatingTags
16201,1541248,Pajak,Bozena,NaN,Linguistics,NaN,0.0,18235935,0.0,average,...,3.0,3.0,4.0,5.0,3.0,1.0,NaN,NaN,NaN,[]


In [26]:
counts.get('26354')

37

In [ ]:
outcome_1, predictors_1 = patsy.dmatrices(" ~ major", df2)
mod_1 = sm.OLS(outcome_1, predictors_1)
res_1 = mod_1.fit()